In [2]:
import torch
import requests
import numpy as np
import pandas as pd
from tqdm import tqdm
from itertools import product
from sklearn.preprocessing import OneHotEncoder
from IPython.display import display, clear_output
from torch.utils.data import Dataset, DataLoader, SequentialSampler, BatchSampler

In [3]:
# import all the documneatation files 

links=pd.read_csv('ml-latest-small/links.csv')
movies=pd.read_csv('ml-latest-small/movies.csv')
ratings=pd.read_csv('ml-latest-small/ratings.csv')
tags=pd.read_csv('ml-latest-small/tags.csv')

In [4]:
links

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
9737,193581,5476944,432131.0
9738,193583,5914996,445030.0
9739,193585,6397426,479308.0
9740,193587,8391976,483455.0


In [5]:
links.movieId.nunique() 

9742

In [6]:
links[links.duplicated(subset=['tmdbId'], keep=False)]

,movieId,imdbId,tmdbId
624,791,113610,NaN
843,1107,102336,NaN
2141,2851,81454,NaN
3027,4051,56600,NaN
4169,6003,290538,4912.0
5532,26587,92337,NaN
5854,32600,377059,NaN
6059,40697,105946,NaN
7382,79299,874957,NaN
9106,144606,270288,4912.0


In [7]:
links.imdbId.nunique() 

9742

In [8]:
links_ch=links.copy()   
# links_ch.drop_duplicates(subset=['tmdbId'], keep='first', inplace=True)
# links_ch.dropna(subset=['tmdbId'], inplace=True)
# links_ch.drop(columns='imdbId',inplace=True)
# links_ch=links_ch.reset_index(drop=True)    
links_ch    

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
9737,193581,5476944,432131.0
9738,193583,5914996,445030.0
9739,193585,6397426,479308.0
9740,193587,8391976,483455.0


In [9]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [10]:
# replace | in column genres with space
movies_ch=movies.copy()
movies_ch['genres'] = movies_ch['genres'].str.replace('|', ' ')

# find all the unique genres 
all_genres=movies_ch['genres'].apply(lambda x: x.split(' ')).explode().unique()


# to exclude the no genra listed
all_genres=all_genres[:-3]

all_genres=np.append(all_genres,'no genres listed')    
all_genres

# add new columns titled all_genres
for i in all_genres:
    movies_ch[i]=0

# fill the columns with the genera 
for i in all_genres:
    movies_ch[i]=movies_ch['genres'].apply(lambda x: 1 if i in x else 0)

# if any of the all_genera columns contains 1 then no  generes listed should be 0
movies_ch["no genres listed"] = movies_ch.iloc[:, 3:-2].apply(lambda row: 0 if (row == 1).any() else 1, axis=1)

# drop the genres column
movies_ch.drop(columns='genres',inplace=True)

# merge the two dataframes
movies_ch=pd.merge(movies_ch,links_ch,how='right',on='movieId')
movies_ch  

#rearange the columns start with tmdbId then the all_genres
movies_ch=movies_ch[['imdbId', 'movieId']+list(all_genres)].reset_index(drop=True)

movies_ch

,imdbId,movieId,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,no genres listed
0,114709,1,1,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,113497,2,1,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,113228,3,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,114885,4,0,0,0,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,113041,5,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9737,5476944,193581,0,1,0,1,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
9738,5914996,193583,0,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9739,6397426,193585,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
9740,8391976,193587,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [11]:
movies_ch.movieId.nunique() 

9742

In [12]:
# movies_ch[movies_ch['title'].apply(lambda x: x.startswith('Toy Story'))]


In [13]:
# add column year of release
# movies_ch['year']=movies_ch['title'].apply(lambda x: x.split(' ')[-1])

In [14]:

# create the category column    
# shape is 9741 (movies only!) * 20 columns(genra)
# catigories=movies_ch.iloc[:,1:]
# catigories

In [15]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [16]:
# convert the timestamp to datetime 

ratings_ch=ratings.copy()

# convert the timestamp to datetime stating from 1970-01-01
# ratings_ch['timestamp_ch'] = pd.to_datetime(ratings_ch['timestamp'], unit='s')

ratings_ch['day_timestamp_rating'] = (ratings_ch['timestamp']/3600).round() # add a new column day
# drop the timestamp column
# ratings_ch.drop(columns='timestamp',inplace=True)
ratings_ch.timestamp = (ratings_ch.timestamp-min(ratings_ch.timestamp)).astype('int64')
ratings_ch.day_timestamp_rating = (ratings_ch.day_timestamp_rating-min(ratings_ch.day_timestamp_rating)).astype('int64')

In [17]:
# ratings_ch.sort_values(by='timestamp_ch',inplace=True)

ratings_ch

,userId,movieId,rating,timestamp,day_timestamp_rating
0,1,1,4.0,136858088,38016
1,1,3,4.0,136856632,38015
2,1,6,4.0,136857609,38016
3,1,47,5.0,136859200,38016
4,1,50,5.0,136858316,38016
...,...,...,...,...,...
100831,610,166534,4.0,665723787,184923
100832,610,168248,5.0,665725476,184923
100833,610,168250,5.0,666148432,185041
100834,610,168252,5.0,665721737,184922


In [18]:
ggr=ratings_ch[["userId","movieId","rating"]].groupby(["userId",'movieId']).count()
ggr

rating
userId movieId        
1      1             1
       3             1
       6             1
       47            1
       50            1
...                ...
610    166534        1
       168248        1
       168250        1
       168252        1
       170875        1

[100836 rows x 1 columns]

In [19]:
# check if the user has rated the movie more than once
ggr.rating.values  
max(ggr.rating.values)

1

In [20]:
ratings_ch[(ratings_ch.movieId==60756) & (ratings_ch.userId==2)]  

,userId,movieId,rating,timestamp,day_timestamp_rating
241,2,60756,5.0,617590365,171552


In [21]:
ratings_ch

,userId,movieId,rating,timestamp,day_timestamp_rating
0,1,1,4.0,136858088,38016
1,1,3,4.0,136856632,38015
2,1,6,4.0,136857609,38016
3,1,47,5.0,136859200,38016
4,1,50,5.0,136858316,38016
...,...,...,...,...,...
100831,610,166534,4.0,665723787,184923
100832,610,168248,5.0,665725476,184923
100833,610,168250,5.0,666148432,185041
100834,610,168252,5.0,665721737,184922


In [22]:
100836+18


100854

In [23]:
ratings_ch.movieId.nunique()
movies_ch.movieId.nunique() 
print('The number of unique movies is',movies_ch.movieId.nunique())
print('The number of unique moives that has at least one rating is',ratings_ch.movieId.nunique())

The number of unique movies is 9742
The number of unique moives that has at least one rating is 9724


In [24]:
# the movies that has norating 
movies[~movies.movieId.isin(ratings_ch.movieId.unique())]
# 18 movie found

,movieId,title,genres
816,1076,"Innocents, The (1961)",Drama|Horror|Thriller
2211,2939,Niagara (1953),Drama|Thriller
2499,3338,For All Mankind (1989),Documentary
2587,3456,"Color of Paradise, The (Rang-e khoda) (1999)",Drama
3118,4194,I Know Where I'm Going! (1945),Drama|Romance|War
4037,5721,"Chosen, The (1981)",Drama
4506,6668,"Road Home, The (Wo de fu qin mu qin) (1999)",Drama|Romance
4598,6849,Scrooge (1970),Drama|Fantasy|Musical
4704,7020,Proof (1991),Comedy|Drama|Romance
5020,7792,"Parallax View, The (1974)",Thriller


In [25]:
# merge the moive_ch and the reting_ch
# how = 'right' to avoid add the no rated moives

all_data = pd.merge (movies_ch, ratings_ch, on='movieId', how = 'right')
all_data

,imdbId,movieId,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,...,Musical,Documentary,IMAX,Western,Film-Noir,no genres listed,userId,rating,timestamp,day_timestamp_rating
0,114709,1,1,1,1,1,1,0,0,0,...,0,0,0,0,0,0,1,4.0,136858088,38016
1,113228,3,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,1,4.0,136856632,38015
2,113277,6,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,4.0,136857609,38016
3,114369,47,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,5.0,136859200,38016
4,114814,50,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,5.0,136858316,38016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100831,4972582,166534,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,610,4.0,665723787,184923
100832,4425200,168248,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,610,5.0,665725476,184923
100833,5052448,168250,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,610,5.0,666148432,185041
100834,3315342,168252,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,610,5.0,665721737,184922


In [26]:
all_data.drop(['timestamp','day_timestamp_rating'],axis=1).to_csv("all_data_ch.csv",index=False)

In [ ]:
sorted_data = all_data.groupby('userId').apply(lambda x: x.sort_values('day_timestamp_rating', ascending=False))
sorted_data.head(25)

C:\Users\AHMED OSAMA\AppData\Local\Temp\ipykernel_8208\3345870625.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sorted_data = all_data.groupby('userId').apply(lambda x: x.sort_values('day_timestamp_rating', ascending=False))


imdbId  movieId  Adventure  Animation  Children  Comedy  Fantasy  \
userId                                                                         
1      161  138987     2492          0          0         0       1        0   
       149  120660     2353          0          0         0       0        0   
       135   77869     2116          1          1         1       0        1   
       136   70016     2137          0          1         1       0        0   
       137   84649     2139          1          1         1       0        0   
       138   90633     2141          1          1         1       1        0   
       144  104257     2268          0          0         0       0        0   
       145  120812     2273          0          0         0       1        0   
       146   99487     2291          0          0         0       0        1   
       147  120586     2329          0          0         0       0        0   
       148  130018     2338          0          0         0       0        0   
       150   24216     2366          1          0         0       0        1   
       132   38969     2099          1          1         1       0        0   
       151  124198     2387          0          0         0       1        0   
       152  155975     2389          0          0         0       0        0   
       154   88011     2406          1          0         0       1        0   
       155   90357     2414          1          0         1       0        1   
       156  120863     2427          0          0         0       0        0   
       157   91225     2450          1          0         0       1        0   
       158   72271     2459          0          0         0       0        0   
       159   90555     2470          1          0         0       1        0   
       160   92086     2478          0          0         0       1        0   
       134   87469     2115          1          0         0       0        1   
       0    114709        1          1          1         1       1        1   
       94   104691     1408          0          0         0       0        0   

            Romance  Drama  Action  ...  Musical  Documentary  IMAX  Western  \
userId                              ...                                        
1      161        1      0       0  ...        0            0     0        0   
       149        0      0       1  ...        0            0     0        0   
       135        0      0       0  ...        0            0     0        0   
       136        0      0       0  ...        0            0     0        0   
       137        0      1       0  ...        0            0     0        0   
       138        0      0       0  ...        0            0     0        0   
       144        0      1       0  ...        0            0     0        0   
       145        0      0       1  ...        0            0     0        0   
       146        1      1       0  ...        0            0     0        0   
       147        0      1       0  ...        0            0     0        0   
       148        0      0       0  ...        0            0     0        0   
       150        0      0       1  ...        0            0     0        0   
       132        0      0       0  ...        1            0     0        0   
       151        0      0       0  ...        0            0     0        0   
       152        0      0       0  ...        0            0     0        0   
       154        1      0       1  ...        0            0     0        0   
       155        0      0       1  ...        0            0     0        0   
       156        0      1       1  ...        0            0     0        0   
       157        0      0       0  ...        0            0     0        0   
       158        0      0       0  ...        0            0     0        0   
       159        0      0       0  ...        0            0     0        0   
       160 

In [ ]:
# number of the users 

all_data.userId.nunique()   

610

In [ ]:
all_data['Train'] = (all_data.groupby(all_data.userId).cumcount(ascending=False) != 0).replace({True:1, False:0})
all_data.rating = all_data.rating.astype(int)
all_data[all_data.userId==2]

C:\Users\AHMED OSAMA\AppData\Local\Temp\ipykernel_8208\4144912804.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  all_data['Train'] = (all_data.groupby(all_data.userId).cumcount(ascending=False) != 0).replace({True:1, False:0})


,imdbId,movieId,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,...,Documentary,IMAX,Western,Film-Noir,no genres listed,userId,rating,timestamp,day_timestamp_rating,Train
232,111161,318,0,0,0,0,0,0,1,0,...,0,0,0,0,0,2,3,617590220,171552,1
233,114694,333,0,0,0,1,0,0,0,0,...,0,0,0,0,0,2,4,617590414,171553,1
234,119217,1704,0,0,0,0,0,1,1,0,...,0,0,0,0,0,2,4,617590613,171553,1
235,172495,3578,1,0,0,0,0,0,1,1,...,0,0,0,0,0,2,4,617590270,171552,1
236,266697,6874,0,0,0,0,0,0,0,1,...,0,0,0,0,0,2,4,617590337,171552,1
237,369339,8798,0,0,0,0,0,0,1,1,...,0,0,0,0,0,2,3,617590345,171552,1
238,415306,46970,0,0,0,1,0,0,0,1,...,0,0,0,0,0,2,4,617590398,171553,1
239,407887,48516,0,0,0,0,0,0,1,0,...,0,0,0,0,0,2,4,617590449,171553,1
240,468569,58559,0,0,0,0,0,0,1,1,...,0,1,0,0,0,2,4,617590526,171553,1
241,838283,60756,0,0,0,1,0,0,0,0,...,0,0,0,0,0,2,5,617590365,171552,1


In [ ]:
all_data_ordered=all_data.groupby('userId').apply(lambda x: x.sort_values(['timestamp','userId'], ascending=True)).reset_index(drop=True)
all_data_ordered.head(10)

C:\Users\AHMED OSAMA\AppData\Local\Temp\ipykernel_8208\1359035833.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  all_data_ordered=all_data.groupby('userId').apply(lambda x: x.sort_values(['timestamp','userId'], ascending=True)).reset_index(drop=True)


,imdbId,movieId,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,...,Documentary,IMAX,Western,Film-Noir,no genres listed,userId,rating,timestamp,day_timestamp_rating,Train
0,117628,804,0,0,0,1,0,1,0,0,...,0,0,0,0,0,1,4,136855884,38015,1
1,86190,1210,1,0,0,0,0,0,0,1,...,0,0,0,0,0,1,5,136855884,38015,1
2,34492,2018,0,1,1,0,0,0,1,0,...,0,0,0,0,0,1,5,136855908,38015,1
3,120915,2628,1,0,0,0,0,0,0,1,...,0,0,0,0,0,1,4,136855908,38015,1
4,120657,2826,1,0,0,0,1,0,0,1,...,0,0,0,0,0,1,4,136855908,38015,1
5,172495,3578,1,0,0,0,0,0,1,1,...,0,0,0,0,0,1,5,136856053,38015,1
6,215129,3617,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,4,136856068,38015,1
7,162650,3744,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,4,136856079,38015,1
8,115734,101,1,0,0,1,0,1,0,0,...,0,0,0,0,0,1,5,136856253,38015,1
9,106677,441,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,4,136856253,38015,1


In [ ]:
# get the catigories shape is 100836 (record user with at least one rating) * 20 (the number of catigories)
catigories=all_data[all_data_ordered.columns[2:-5]]
catigories

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,no genres listed
0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100831,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0
100832,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0
100833,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
100834,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0


In [ ]:
all_data_train=all_data_ordered[all_data_ordered.Train==1].copy()
all_data_train["oness"]=1

rating_history= pd.pivot_table(all_data_train, index='userId', columns='imdbId', values='oness',fill_value=0).astype(int)
rating_history

imdbId,417,439,516,4972,6333,6864,7264,7361,8133,10040,...,7379330,7387408,7539884,7544820,7620650,7681902,7738550,7807952,7808620,7924798
userId,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
607,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
608,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
all_data_train.movieId.nunique()

9665

In [ ]:
rating_history_norm=rating_history/rating_history.values.sum(axis=1, keepdims=True)
rating_history_norm

imdbId,417,439,516,4972,6333,6864,7264,7361,8133,10040,...,7379330,7387408,7539884,7544820,7620650,7681902,7738550,7807952,7808620,7924798
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
rating_history_norm.loc[2]

imdbId
417        0.0
439        0.0
516        0.0
4972       0.0
6333       0.0
          ... 
7681902    0.0
7738550    0.0
7807952    0.0
7808620    0.0
7924798    0.0
Name: 2, Length: 9665, dtype: float64

In [ ]:
for i in rating_history_norm.loc[2].values:
    if i!=0.0:
        print(i)


0.03571428571428571
0.03571428571428571
0.03571428571428571
0.03571428571428571
0.03571428571428571
0.03571428571428571
0.03571428571428571
0.03571428571428571
0.03571428571428571
0.03571428571428571
0.03571428571428571
0.03571428571428571
0.03571428571428571
0.03571428571428571
0.03571428571428571
0.03571428571428571
0.03571428571428571
0.03571428571428571
0.03571428571428571
0.03571428571428571
0.03571428571428571
0.03571428571428571
0.03571428571428571
0.03571428571428571
0.03571428571428571
0.03571428571428571
0.03571428571428571
0.03571428571428571


In [ ]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [ ]:
last_movie=all_data_ordered[['userId','imdbId','rating','timestamp','day_timestamp_rating']].copy()
last_movie

,userId,imdbId,rating,timestamp,day_timestamp_rating
0,1,117628,4,136855884,38015
1,1,86190,5,136855884,38015
2,1,34492,5,136855908,38015
3,1,120915,4,136855908,38015
4,1,120657,4,136855908,38015
...,...,...,...,...,...
100831,610,1288558,3,667834654,185509
100832,610,116367,4,667834667,185509
100833,610,114608,3,667834684,185509
100834,610,72271,3,667834790,185509


In [ ]:
last_movie["last_seen"] = last_movie["imdbId"].copy()

for i in last_movie.userId.unique():
    small_list = last_movie[last_movie.userId == i][['imdbId']][:-1]
    zero_row = pd.DataFrame([[0]], columns=['imdbId'])
    new_seq = pd.concat([zero_row, small_list], ignore_index=True)
    last_movie.loc[last_movie.userId == i, 'last_seen'] = new_seq['imdbId'].values

last_movie

,userId,imdbId,rating,timestamp,day_timestamp_rating,last_seen
0,1,117628,4,136855884,38015,0
1,1,86190,5,136855884,38015,117628
2,1,34492,5,136855908,38015,86190
3,1,120915,4,136855908,38015,34492
4,1,120657,4,136855908,38015,120915
...,...,...,...,...,...,...
100831,610,1288558,3,667834654,185509,93605
100832,610,116367,4,667834667,185509,1288558
100833,610,114608,3,667834684,185509,116367
100834,610,72271,3,667834790,185509,114608


In [ ]:
last_movie[last_movie.userId == 2][['imdbId','last_seen']]  

,imdbId,last_seen
232,111161,0
233,1375666,111161
234,4299972,1375666
235,470752,4299972
236,1853728,470752
237,2582802,1853728
238,172495,2582802
239,1345836,172495
240,1130884,1345836
241,1568346,1130884


In [ ]:
all_data_ordered["last_seen"]=last_movie["last_seen"].copy()

In [ ]:
all_data_ordered.drop(columns='day_timestamp_rating',inplace=True)
all_data_ordered

,imdbId,movieId,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,...,Documentary,IMAX,Western,Film-Noir,no genres listed,userId,rating,timestamp,Train,last_seen
0,117628,804,0,0,0,1,0,1,0,0,...,0,0,0,0,0,1,4,136855884,1,0
1,86190,1210,1,0,0,0,0,0,0,1,...,0,0,0,0,0,1,5,136855884,1,117628
2,34492,2018,0,1,1,0,0,0,1,0,...,0,0,0,0,0,1,5,136855908,1,86190
3,120915,2628,1,0,0,0,0,0,0,1,...,0,0,0,0,0,1,4,136855908,1,34492
4,120657,2826,1,0,0,0,1,0,0,1,...,0,0,0,0,0,1,4,136855908,1,120915
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100831,1288558,101739,0,0,0,0,0,0,0,0,...,0,0,0,0,0,610,3,667834654,1,93605
100832,116367,70,0,0,0,1,0,0,0,1,...,0,0,0,0,0,610,4,667834667,1,1288558
100833,114608,328,0,0,0,0,0,0,0,0,...,0,0,0,0,0,610,3,667834684,1,116367
100834,72271,2459,0,0,0,0,0,0,0,0,...,0,0,0,0,0,610,3,667834790,1,114608


In [ ]:
user_movie_lastseen=last_movie[['userId','imdbId','last_seen']].copy()
user_movie_lastseen 

,userId,imdbId,last_seen
0,1,117628,0
1,1,86190,117628
2,1,34492,86190
3,1,120915,34492
4,1,120657,120915
...,...,...,...
100831,610,1288558,93605
100832,610,116367,1288558
100833,610,114608,116367
100834,610,72271,114608


In [ ]:
user_movie_lastseen[user_movie_lastseen.userId==2]

,userId,imdbId,last_seen
232,2,111161,0
233,2,1375666,111161
234,2,4299972,1375666
235,2,470752,4299972
236,2,1853728,470752
237,2,2582802,1853728
238,2,172495,2582802
239,2,1345836,172495
240,2,1130884,1345836
241,2,1568346,1130884


In [ ]:
print("the unique number of the users is ", user_movie_lastseen.userId.nunique())   
print("the unique number of the movies is ", user_movie_lastseen.imdbId.nunique())

the unique number of the users is  610
the unique number of the movies is  9724


In [ ]:
9724*2+610

20058

In [ ]:
active_ohe = OneHotEncoder(sparse_output=False).fit(user_movie_lastseen)

In [ ]:
sample_df = pd.DataFrame({'userId': [610], 'imdbId': [111161] , 'last_seen': [0]})
sample_df

pd.DataFrame(
    active_ohe.transform(
        sample_df[active_ohe.feature_names_in_]
        ),
    columns=active_ohe.get_feature_names_out()
    )

,userId_1,userId_2,userId_3,userId_4,userId_5,userId_6,userId_7,userId_8,userId_9,userId_10,...,last_seen_7387408,last_seen_7539884,last_seen_7544820,last_seen_7681902,last_seen_7690670,last_seen_7738550,last_seen_7807952,last_seen_7808620,last_seen_7924798,last_seen_8391976
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
for i in active_ohe.get_feature_names_out():
    if i.split("_")[-1]=="0":
        print(i)


last_seen_0


In [ ]:
all_data_ordered

,imdbId,movieId,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,...,Documentary,IMAX,Western,Film-Noir,no genres listed,userId,rating,timestamp,Train,last_seen
0,117628,804,0,0,0,1,0,1,0,0,...,0,0,0,0,0,1,4,136855884,1,0
1,86190,1210,1,0,0,0,0,0,0,1,...,0,0,0,0,0,1,5,136855884,1,117628
2,34492,2018,0,1,1,0,0,0,1,0,...,0,0,0,0,0,1,5,136855908,1,86190
3,120915,2628,1,0,0,0,0,0,0,1,...,0,0,0,0,0,1,4,136855908,1,34492
4,120657,2826,1,0,0,0,1,0,0,1,...,0,0,0,0,0,1,4,136855908,1,120915
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100831,1288558,101739,0,0,0,0,0,0,0,0,...,0,0,0,0,0,610,3,667834654,1,93605
100832,116367,70,0,0,0,1,0,0,0,1,...,0,0,0,0,0,610,4,667834667,1,1288558
100833,114608,328,0,0,0,0,0,0,0,0,...,0,0,0,0,0,610,3,667834684,1,116367
100834,72271,2459,0,0,0,0,0,0,0,0,...,0,0,0,0,0,610,3,667834790,1,114608


In [ ]:

all_data_ordered_to_split=all_data_ordered[['userId','imdbId','last_seen','timestamp']+list(all_genres)+['rating','Train']].copy()
all_data_ordered_to_split

,userId,imdbId,last_seen,timestamp,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,no genres listed,rating,Train
0,1,117628,0,136855884,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,4,1
1,1,86190,117628,136855884,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,5,1
2,1,34492,86190,136855908,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,5,1
3,1,120915,34492,136855908,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,4,1
4,1,120657,120915,136855908,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100831,610,1288558,93605,667834654,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,1
100832,610,116367,1288558,667834667,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,4,1
100833,610,114608,116367,667834684,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,1
100834,610,72271,114608,667834790,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,1


In [ ]:
train_data=all_data_ordered_to_split[all_data_ordered_to_split.Train==1]
test_data=all_data_ordered_to_split[all_data_ordered_to_split.Train==0]

print("the dimensions of the train data is ", train_data.shape[0],"x",train_data.shape[1])
print("the dimensions of the test data is ", test_data.shape[0],"x",test_data.shape[1])

the dimensions of the train data is  100226 x 26
the dimensions of the test data is  610 x 26


In [ ]:
all_genres

array(['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Mystery', 'Sci-Fi', 'War', 'Musical', 'Documentary', 'IMAX',
       'Western', 'Film-Noir', 'no genres listed'], dtype=object)

In [ ]:
users_movies=all_data_ordered_to_split[["userId","imdbId","last_seen","rating"]+list(all_genres)]

In [ ]:
users_movies[users_movies.imdbId==unique_movies[2]]['rating'].values.mean()

3.1842105263157894

In [ ]:

unique_movies=users_movies['imdbId'].unique()
average_rating = []
for i in unique_movies: 
    average_rating.append(users_movies[users_movies.imdbId==i]['rating'].values.mean())

In [ ]:
average_rating_df=pd.DataFrame({'imdbId':unique_movies,'average_rating':average_rating})

In [ ]:
average_rating_df

,imdbId,average_rating
0,117628,3.250000
1,86190,4.025510
2,34492,3.184211
3,120915,2.971429
4,120657,2.730769
...,...,...
9719,259974,3.000000
9720,116404,2.000000
9721,91083,4.000000
9722,780607,3.000000


In [ ]:
users_movies= users_movies.merge(average_rating_df, on='imdbId', how='left')
users_movies


,userId,imdbId,last_seen,rating,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,no genres listed,average_rating
0,1,117628,0,4,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,3.250000
1,1,86190,117628,5,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,4.025510
2,1,34492,86190,5,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,3.184211
3,1,120915,34492,4,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,2.971429
4,1,120657,120915,4,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,2.730769
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100831,610,1288558,93605,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3.000000
100832,610,116367,1288558,4,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,3.400000
100833,610,114608,116367,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3.000000
100834,610,72271,114608,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3.153846


In [ ]:
users_movies

,userId,imdbId,last_seen,rating,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,no genres listed,average_rating
0,1,117628,0,4,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,3.250000
1,1,86190,117628,5,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,4.025510
2,1,34492,86190,5,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,3.184211
3,1,120915,34492,4,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,2.971429
4,1,120657,120915,4,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,2.730769
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100831,610,1288558,93605,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3.000000
100832,610,116367,1288558,4,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,3.400000
100833,610,114608,116367,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3.000000
100834,610,72271,114608,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3.153846


In [ ]:
# all_data_ordered_to_split[all_data_ordered_to_split.userId==2]

In [ ]:
# save all data ss


# train_data.to_csv("train_data.csv",index=False)
# test_data.to_csv("test_data.csv",index=False)   
# rating_history_norm.astype(int).to_csv("rating_history_norm.csv",index=True)
# user_movie_lastseen.to_csv("user_movie_lastseen.csv",index=False)   
users_movies.to_csv("user_movies.csv",index=False)

# save the onehot encoding
import pickle

with open('one_hot_encoder_sec.pkl', 'wb') as f:
    pickle.dump(active_ohe, f)
    

In [ ]:
test_data

,userId,imdbId,last_seen,timestamp,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,no genres listed,rating,Train
218,1,66026,108052,136859387,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,5,0
234,2,4299972,1375666,617590236,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,5,0
289,3,1190080,78087,478339549,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
503,4,283509,199129,179444809,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,5,0
547,5,116282,103064,19310643,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98477,606,1446192,837562,540335701,1,1,1,0,1,0,...,0,0,0,0,1,0,0,0,3,0
98516,607,66026,107048,134954516,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,3,0
99461,608,829482,379786,361244110,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,4,0
99533,609,78875,116722,19096465,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,4,0


In [ ]:
ww

NameError: name 'ww' is not defined

In [ ]:
# read all data 

train_data=pd.read_csv("train_data.csv",index_col=False)
test_data=pd.read_csv("test_data.csv",index_col=False)
rating_history_norm=pd.read_csv("rating_history_norm.csv",index_col=0)
user_movie_lastseen=pd.read_csv("user_movie_lastseen.csv",index_col=False)


# import the one hot encoder
import pickle
with open('one_hot_encoder_sec.pkl', 'rb') as f:
    active_ohe = pickle.load(f)


In [ ]:
rating_history_norm

,417,439,516,4972,6333,6864,7264,7361,8133,10040,...,7379330,7387408,7539884,7544820,7620650,7681902,7738550,7807952,7808620,7924798
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
train_data


,userId,imdbId,last_seen,timestamp,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,no genres listed,rating,Train
0,1,117628,0,136855884,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,4,1
1,1,86190,117628,136855884,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,5,1
2,1,34492,86190,136855908,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,5,1
3,1,120915,34492,136855908,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,4,1
4,1,120657,120915,136855908,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100831,610,1288558,93605,667834654,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,1
100832,610,116367,1288558,667834667,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,4,1
100833,610,114608,116367,667834684,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,1
100834,610,72271,114608,667834790,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,1


In [ ]:
test_data

,userId,imdbId,last_seen,timestamp,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,no genres listed,rating,Train
218,1,66026,108052,136859387,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,5,0
234,2,4299972,1375666,617590236,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,5,0
289,3,1190080,78087,478339549,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
503,4,283509,199129,179444809,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,5,0
547,5,116282,103064,19310643,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98477,606,1446192,837562,540335701,1,1,1,0,1,0,...,0,0,0,0,1,0,0,0,3,0
98516,607,66026,107048,134954516,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,3,0
99461,608,829482,379786,361244110,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,4,0
99533,609,78875,116722,19096465,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,4,0


In [ ]:
class DPMovieDataset(Dataset):
  # dataset_train = DPMovieDataset(user_mappings.values, train, agg_history_norm, active_ohe)

  def __init__(self, user_ids:np.ndarray, data: pd.DataFrame, agg_hist: pd.DataFrame, active_encoder: OneHotEncoder, recommendation: bool=False):
    self.user_ids = user_ids
    self.data = data
    self.agg_hist = agg_hist
    self.active_encoder = active_encoder
    self.recommendation = recommendation

  def __len__(self):
    return self.user_ids.shape[0]

  def __getitem__(self, idx):
    # batch_data = self.data.iloc[self.data.userid.values isin  self.user_ids[idx]]
    batch_data = self.data[self.data['userId'].isin(self.user_ids[idx].flatten())]

    try:
        cat_cols = batch_data.drop(columns=['userId', 'rating', 'imdbId', 'Train', 'last_seen','timestamp'])
    except:
        cat_cols = batch_data.drop(columns=['userId', 'rating', 'imdbId', 'Train', 'last_seen','timestamp'])
    
    agg_history = batch_data[['userId']].merge(self.agg_hist, left_on='userId', right_index=True) 
    # Get the aggregated history for each selected transaction using merge
    
    active_groups = self.active_encoder.transform(batch_data[['userId','imdbId','last_seen']]) 
    # Use active_encoder to generate the active columns for user/item pairs available in batch_data

    features = torch.from_numpy(np.hstack((active_groups, agg_history.iloc[:,1:].values, cat_cols))) 
    # Concatenate the processed columns together horizontally

    if not self.recommendation:
      targets = batch_data['rating']
      return features, targets , features.shape[1]
    else:
      return features


In [ ]:
dataset_train = DPMovieDataset(pd.DataFrame(rating_history_norm.index).values, train_data, rating_history_norm, active_ohe)
dataset_test = DPMovieDataset(pd.DataFrame(rating_history_norm.index).values, test_data, rating_history_norm, active_ohe)


dataloader_train = DataLoader(dataset_train,
                              sampler=BatchSampler(SequentialSampler(dataset_train), batch_size=20, drop_last=False),
                              batch_size=None)

dataloader_test = DataLoader(dataset_test,
                              sampler=BatchSampler(SequentialSampler(dataset_test), batch_size=20, drop_last=False),
                              batch_size=None)

In [ ]:

batch_data__ = train_data[train_data['userId'].isin(pd.DataFrame(rating_history_norm.index).values[[0]].flatten())]
batch_data__

# cat_cols__ = batch_data__.drop(columns=['userId', 'rating', 'imdbId', 'Train', 'last_seen'])
# cat_cols__

# agg_history__ = batch_data__[['userId']].merge(rating_history_norm, left_on='userId', right_index=True) 
# agg_history__

# active_groups__ = active_ohe.transform(batch_data__[['userId','imdbId','last_seen']]) 
# active_groups__.shape

# features = np.hstack((active_groups__, agg_history__.iloc[:,1:].values, cat_cols__))
# features.shape

,userId,imdbId,last_seen,timestamp,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,no genres listed,rating,Train
0,1,117628,0,136855884,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,4,1
1,1,86190,117628,136855884,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,5,1
2,1,34492,86190,136855908,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,5,1
3,1,120915,34492,136855908,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,4,1
4,1,120657,120915,136855908,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,1,119640,109370,136859497,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,3,1
228,1,108358,119640,136859538,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,5,1
229,1,92086,108358,136859554,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,4,1
230,1,99088,92086,136859561,1,0,0,1,0,0,...,1,0,0,0,0,1,0,0,4,1


In [ ]:
# Hint: This is how you can check the no. features in our dataset
# x represents the features of a single data point
# y represents the target of a single data point
for x, y ,length in dataloader_train:
  print(x.shape)

torch.Size([3199, 29721])
torch.Size([2419, 29721])
torch.Size([3074, 29721])
torch.Size([3932, 29721])
torch.Size([2724, 29721])
torch.Size([3305, 29721])
torch.Size([2898, 29721])
torch.Size([1952, 29721])
torch.Size([2291, 29721])
torch.Size([3274, 29721])
torch.Size([2971, 29721])
torch.Size([3429, 29721])
torch.Size([2398, 29721])
torch.Size([3293, 29721])
torch.Size([4070, 29721])
torch.Size([4369, 29721])
torch.Size([2470, 29721])
torch.Size([2218, 29721])
torch.Size([3430, 29721])
torch.Size([3083, 29721])
torch.Size([4374, 29721])
torch.Size([2659, 29721])
torch.Size([3225, 29721])
torch.Size([5526, 29721])
torch.Size([2910, 29721])
torch.Size([2154, 29721])
torch.Size([2140, 29721])
torch.Size([2235, 29721])
torch.Size([3150, 29721])
torch.Size([6092, 29721])
torch.Size([4962, 29721])


In [ ]:
# the length of the features is
length

29722

In [ ]:
class FactorizationMachine(torch.nn.Module):
    def __init__(self, n, k, bias=False):
        super(FactorizationMachine, self).__init__()
        self.n = n
        self.k = k
        self.linear = torch.nn.Linear(self.n, 1, bias)
        self.V = torch.nn.Parameter(torch.randn(n, k))  # Creating the latent matrix V of size (n X k) and initializing it with random values

    def forward(self, x_batch):
        x_batch = x_batch.float()

        # Perform the first part of the interaction term: row-wise-sum((XV)^2)
        part_1 = torch.sum(torch.mm(x_batch, self.V).pow(2), dim=1, keepdim=True)

        # Perform the second part of the interaction term: row-wise-sum((X)^2 * (V)^2))
        part_2 = torch.sum(torch.mm(x_batch.pow(2), self.V.pow(2)), dim=1, keepdim=True)

        # Put the interaction term parts together
        inter_term = 0.5 * (part_1 - part_2)

        # Perform the linear part of the model equation
        var_strength = self.linear(x_batch)

        return var_strength + inter_term

In [ ]:
model = FactorizationMachine(n=length, k=20)

In [ ]:
def model_step(model, x, y=None, optimizer=None, train=True):
  if train: # If we're in training phase, then zero the gradients and make sure the model is set to train
    model.train()
    optimizer.zero_grad()
  else: # If we're in evaluation phase, then make sure the model is set to eval
    model.eval()

  with torch.set_grad_enabled(train): # Either to perform the next lines with gradient tracing or not
    pred = model(x).squeeze()
    # Get the model output from x

    pred = pred.reshape(pred.shape[0], ) 
    # Flatten the prediction values

  y = torch.from_numpy(y.values.reshape(y.shape[0], )).float()

  criterion = torch.nn.MSELoss()
  
  # Define the criterion as MSELoss from torch
  loss = criterion(pred, y)

  if train:
    loss.backward()
    optimizer.step()

  return loss

In [ ]:
def train_loop(model, train_loader, eval_loader, lr, w_decay, epochs, eval_step):
  step = 0
  """ Defining our optimizer """
  optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=w_decay)
  epochs_l, steps, t_losses, v_losses = [], [], [], []

  epochs_tqdm = tqdm(range(epochs), desc='Training in Progress', leave=True)
  for epoch in epochs_tqdm:
    for x, y ,length in train_loader:
      loss_batch = model_step(model, x, y, optimizer, train=True)
      step +=1
      if step % eval_step == 0:
        train_loss = loss_batch
        val_loss = 0

        with torch.no_grad():
          for x, y,length in eval_loader:
              val_loss += model_step(model, x, y, train=False)

        # for x, y, length in eval_loader:
        #   val_loss += model_step(model, x, y, train=False)


        epochs_l.append(epoch+1)
        steps.append(step)
        t_losses.append(train_loss.detach().numpy())
        v_losses.append(val_loss.detach().numpy())
        
        clear_output(wait=True)
        display(pd.DataFrame({'Epoch': epochs_l, 'Step': step,'Training Loss': t_losses, 'Validation Loss': v_losses}))
  
  return model


In [ ]:
# train_loop(model, dataloader_train, dataloader_test, lr=0.004, w_decay=0.0003, epochs=100, eval_step=100)

,Epoch,Step,Training Loss,Validation Loss
0,4,3100,113.5757,4021.8765
1,7,3100,73.014465,2598.0994
2,10,3100,32.686993,1874.7462
3,13,3100,23.717236,1440.1234
4,17,3100,28.003862,1151.0706
5,20,3100,17.80892,946.4534
6,23,3100,9.676678,793.99756
7,26,3100,9.632179,676.9579
8,30,3100,7.4485793,584.25635
9,33,3100,5.108015,509.32553


Training in Progress: 100%|██████████| 100/100 [2:01:34<00:00, 72.95s/it]


FactorizationMachine(
  (linear): Linear(in_features=29721, out_features=1, bias=False)
)

In [ ]:
import torch

# Save the model
# torch.save(model.state_dict(), 'model.pt')

In [ ]:
# import the model 

# model=FactorizationMachine(n=length, k=5)
model.load_state_dict(torch.load('model.pt'))

In [ ]:

model.eval()
with torch.no_grad():

    user_index = 92
    user_data = dataset_test[user_index] 
    user_features = user_data[0] 
    
    user_features_tensor = torch.tensor(user_features) 
    
    # Make predictions
    predicted_rating = model(user_features_tensor)
    predicted_rating = predicted_rating.item()
    
    print(f'Predicted rating for User of interest: {predicted_rating}')


    actual_rating = user_data[1] 
    print(f'Actual Rating: {user_data[1].to_numpy()[0]}')

Predicted rating for User of interest: 4.509391784667969
Actual Rating: 4


C:\Users\AHMED OSAMA\AppData\Local\Temp\ipykernel_19132\1205500658.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  user_features_tensor = torch.tensor(user_features)


In [ ]:
dataloader_test.__len__()

31

In [ ]:
ran

31

In [ ]:


def rmse_user_to_pred(dataloader_test):
    pred=[]
    act=[]
    model.eval()
    with torch.no_grad():

        for i in range(610):

            user_features = dataset_test[i][0] 
            user_features_tensor = torch.tensor(user_features) 
            predicted_rating = model(user_features_tensor).item()
            pred.append(predicted_rating)
            act.append(dataset_test[i][1].item() )

        pred = np.array(pred)
        act = np.array(act)

        # Compute RMSE
        rmse = np.sqrt(np.mean((pred - act) ** 2))

        # Compute R^2 score
        ss_res = np.sum((act - pred) ** 2)
        ss_tot = np.sum((act - np.mean(act)) ** 2)
        r2_score = 1 - (ss_res / ss_tot)

    return rmse, r2_score , pred, act

In [ ]:
a,b,c,d=rmse_user_to_pred(dataloader_test)